# Sciunit Cross-OS Reproducibility Demo on Chameleon

1. Create an **Ubuntu VM** to author a Sciunit experiment
2. Create a **Fedora VM** to reproduce the experiment
3. Use SSH to interact with both machines
4. Demonstrate cross-OS reproducibility using Sciunit

This notebook focuses only on **infrastructure setup**.
Sciunit commands are executed directly on the VMs


In [ ]:
import chi, os, time, datetime
from chi import lease
from chi import server
from chi import context
from chi import network

context.version = "1.0"
context.choose_project()
context.choose_site(default="KVM@TACC")

username = os.getenv("USER")


In [ ]:
exp_name = "sciunit_cross_os_demo"

ubuntu_server_name = f"{exp_name}-ubuntu-{username}"
centOS_server_name = f"{exp_name}-centOS-{username}"
lease_name = f"{exp_name}-lease-{username}"

print("Experiment name:", exp_name)


In [ ]:
l = lease.Lease(
    lease_name,
    duration=datetime.timedelta(hours=6)
)

l.add_flavor_reservation(
    id=server.get_flavor_id("m1.small"),
    amount=2
)

l.submit(idempotent=True)
l.show()


In [ ]:
!mkdir -p ~/ssh_keys
!chmod 700 ~/ssh_keys
!mv /work/P_Key_NASA_DEMO.pem ~/ssh_keys/
!chmod 600 ~/ssh_keys/P_Key_NASA_DEMO.pem

!ls -ld ~/ssh_keys
!ls -l ~/ssh_keys/P_Key_NASA_DEMO.pem

In [ ]:
ubuntu_image = "CC-Ubuntu22.04"

ubuntu_vm = server.Server(
    name=ubuntu_server_name,
    image_name=ubuntu_image,
    flavor_name=l.get_reserved_flavors()[0].name,
    key_name="P_Key_NASA_DEMO"
)

ubuntu_vm.submit(idempotent=True)


In [ ]:
centOS_image = "CC-CentOS9-Stream-20230518"

centOS_vm = server.Server(
    name=centOS_server_name,
    image_name=centOS_image,
    flavor_name=l.get_reserved_flavors()[0].name,
    key_name="P_Key_NASA_DEMO"
)

centOS_vm.submit(idempotent=True)


In [ ]:
ubuntu_vm.associate_floating_ip()
ubuntu_ip = ubuntu_vm.get_floating_ip()
print("Ubuntu VM IP:", ubuntu_ip)

In [ ]:
centOS_vm.associate_floating_ip()
centOS_ip = centOS_vm.get_floating_ip()
print("centOS VM IP:", centOS_ip)

In [ ]:
sg_list = network.list_security_groups(name_filter="allow-ssh")

if sg_list:
    sg = sg_list[0]
else:
    sg = network.SecurityGroup({
        "name": "allow-ssh",
        "description": "Enable SSH access"
    })
    sg.add_rule("ingress", "tcp", 22)
    sg.submit()

ubuntu_vm.add_security_group(sg.id)
centOS_vm.add_security_group(sg.id)


In [ ]:
ubuntu_vm.check_connectivity()
centOS_vm.check_connectivity()

print("Both Ubuntu and centOS VMs are ready.")


In [ ]:
import paramiko

HOST = ubuntu_ip   # replace with your VM IP
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(
    hostname=HOST,
    username=USER,
    key_filename=KEY_PATH,
    timeout=30,
    allow_agent=False,
    look_for_keys=False,
)

stdin, stdout, stderr = ssh.exec_command("whoami")
print(stdout.read().decode())

ssh.close()


In [ ]:
import paramiko

HOST = centOS_ip   # replace with your VM IP
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(
    hostname=HOST,
    username=USER,
    key_filename=KEY_PATH,
    timeout=30,
    allow_agent=False,
    look_for_keys=False,
)

stdin, stdout, stderr = ssh.exec_command("whoami")
print(stdout.read().decode())

ssh.close()


In [ ]:
import paramiko

HOST = ubuntu_ip
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

SCRIPT_APT_PIP = r'''
set -e

echo "==== STEP 1: apt update ===="
sudo apt update -y
echo "✅ OK: apt update"

echo "==== STEP 2: install python3-pip ===="
sudo apt install -y python3-pip
python3 -m pip --version
echo "✅ OK: pip installed"
exit 0
'''

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(
    hostname=HOST,
    username=USER,
    key_filename=KEY_PATH,
    timeout=30,
    allow_agent=False,
    look_for_keys=False,
)

stdin, stdout, stderr = ssh.exec_command(
    f"bash -lc '{SCRIPT_APT_PIP}'",
    get_pty=True
)

for line in iter(stdout.readline, ""):
    print(line, end="")

err = stderr.read().decode()
if err:
    print(err)
    raise RuntimeError("APT / pip setup failed")

ssh.close()


In [ ]:
import paramiko

HOST = ubuntu_ip
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

SCRIPT_SCIUNIT = r'''
set -e

echo "==== STEP 4: clone sciunit ===="
cd ~
if [ ! -d "$HOME/sciunit" ]; then
  git clone https://github.com/radiant-systems-lab/sciunit.git
else
  echo "Sciunit repo already exists, skipping clone"
fi
echo "✅ OK: sciunit ready"

echo "==== STEP 5: checkout nasa-demo ===="
cd ~/sciunit
git fetch
git checkout nasa-demo
echo "✅ OK: nasa-demo branch"

echo "==== STEP 6: install cmake ===="
sudo apt install -y cmake
cmake --version
echo "✅ OK: cmake installed"

echo "==== STEP 7: install sciunit ===="
pip install .
echo "✅ OK: sciunit installed"

echo "==== STEP 8: update PATH (session only) ===="
export PATH="$HOME/.local/bin:$PATH"
echo "PATH now is: $PATH"
echo "✅ OK: PATH updated for this session"
exit 0

'''

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(
    hostname=HOST,
    username=USER,
    key_filename=KEY_PATH,
    timeout=30,
    allow_agent=False,
    look_for_keys=False,
)

stdin, stdout, stderr = ssh.exec_command(
    f"bash -lc '{SCRIPT_SCIUNIT}'",
    get_pty=True
)

for line in iter(stdout.readline, ""):
    print(line, end="")

err = stderr.read().decode()
if err:
    print(err)
    raise RuntimeError("Sciunit setup failed")

ssh.close()


In [ ]:
import paramiko

HOST = ubuntu_ip                      # MUST be floating IP 
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

SETUP_SCRIPT = r'''
set -e

echo "==== STEP 9: download miniconda ===="
cd ~
if [ ! -f Miniconda3-latest-Linux-x86_64.sh ]; then
  wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
fi
echo "✅ OK: miniconda downloaded"

echo "==== STEP 10: install miniconda ===="
if [ ! -d "$HOME/miniconda3" ]; then
  bash Miniconda3-latest-Linux-x86_64.sh -b -p $HOME/miniconda3
fi
echo "✅ OK: miniconda installed"

echo "==== STEP 11: load conda into current shell ===="
source "$HOME/miniconda3/etc/profile.d/conda.sh"
echo "✅ OK: conda available"

echo "==== STEP 11.1: accept conda Terms of Service ===="
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r
echo "✅ OK: conda ToS accepted"

echo "==== STEP 12: create python 3.6 env ===="
if ! conda env list | grep -q '^py36'; then
  conda create -y -n py36 python=3.6
fi
echo "✅ OK: python 3.6 env created"

echo "==== STEP 13: activate py36 ===="
conda activate py36
echo "✅ OK: py36 activated"

echo "==== STEP 14: install scientific libs ===="
conda install -y numpy pandas matplotlib scipy cartopy
echo "✅ OK: conda scientific libs installed"

echo "==== STEP 15: install pyhdf ===="
pip install pyhdf
echo "✅ OK: pyhdf installed"

echo "==== STEP 16: verify imports ===="
python - << 'EOF'
import numpy, pandas, matplotlib, scipy, cartopy, pyhdf
print("ALL IMPORTS SUCCESSFUL")
EOF
echo "✅ OK: imports verified"

echo "==== STEP 17: enable conda for this session only ===="
source "$HOME/miniconda3/etc/profile.d/conda.sh"
conda activate py36
echo "Conda env active: $CONDA_DEFAULT_ENV"
echo "🎉 ALL STEPS COMPLETED SUCCESSFULLY"
exit 0

'''

# ---------------- SSH EXECUTION ----------------

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(
    hostname=HOST,
    username=USER,
    key_filename=KEY_PATH,
    timeout=30,
    allow_agent=False,
    look_for_keys=False,
)

stdin, stdout, stderr = ssh.exec_command(
    f"bash -lc '{SETUP_SCRIPT}'",
    get_pty=True
)

# Stream output live (prevents “Paramiko is slow” illusion)
for line in iter(stdout.readline, ""):
    print(line, end="")

err = stderr.read().decode()
if err:
    print(err)
    raise RuntimeError("Remote setup failed")

ssh.close()


### For Cento Machine

In [ ]:
import paramiko

HOST = centOS_ip          # floating IP
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

SCRIPT_SYS = r'''
set -e

echo "==== STEP 1: system update ===="
sudo dnf -y update
echo "✅ OK: system updated"

echo "==== STEP 2: verify python & pip ===="
python3 --version
pip3 --version
echo "✅ OK: python & pip verified"

exit 0
'''

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(HOST, username=USER, key_filename=KEY_PATH, timeout=30)

stdin, stdout, stderr = ssh.exec_command(
    f"bash -lc '{SCRIPT_SYS}'",
    get_pty=True
)

for line in iter(stdout.readline, ""):
    print(line, end="")

err = stderr.read().decode()
if err:
    print(err)
    raise RuntimeError("CentOS system setup failed")

ssh.close()


In [ ]:
import paramiko

HOST = centOS_ip
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

SCRIPT_SCIUNIT = r'''
set -e

echo "==== STEP 4: clone sciunit ===="
cd ~
if [ ! -d "$HOME/sciunit" ]; then
  git clone https://github.com/radiant-systems-lab/sciunit.git
else
  echo "Sciunit repo already exists"
fi
echo "✅ OK: sciunit repo ready"

echo "==== STEP 5: checkout nasa-demo ===="
cd ~/sciunit
sudo dnf install -y cmake
git checkout nasa-demo
echo "✅ OK: nasa-demo branch"

echo "==== STEP 6: install sciunit ===="
pip3 install --user .
echo "✅ OK: sciunit installed"

echo "==== STEP 7: update PATH ===="
grep -qxF 'export PATH=$HOME/.local/bin:$PATH' ~/.bashrc || \
  echo 'export PATH=$HOME/.local/bin:$PATH' >> ~/.bashrc
echo "✅ OK: PATH updated"

exit 0
'''

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(HOST, username=USER, key_filename=KEY_PATH, timeout=30)

stdin, stdout, stderr = ssh.exec_command(
    f"bash -lc '{SCRIPT_SCIUNIT}'",
    get_pty=True
)

for line in iter(stdout.readline, ""):
    print(line, end="")

err = stderr.read().decode()
if err:
    print(err)
    raise RuntimeError("Sciunit install failed")

ssh.close()



In [ ]:
import paramiko

HOST = centOS_ip
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

SCRIPT_CONDA = r'''
set -e

echo "==== STEP 8: download miniconda ===="
cd ~
if [ ! -f Miniconda3-latest-Linux-x86_64.sh ]; then
  wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
fi
echo "✅ OK: miniconda downloaded"

echo "==== STEP 9: install miniconda ===="
if [ ! -d "$HOME/miniconda3" ]; then
  bash Miniconda3-latest-Linux-x86_64.sh -b -p $HOME/miniconda3
fi
echo "✅ OK: miniconda installed"

echo "==== STEP 10: load conda ===="
source "$HOME/miniconda3/etc/profile.d/conda.sh"
echo "✅ OK: conda loaded"

echo "==== STEP 11: accept conda ToS ===="
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r
echo "✅ OK: ToS accepted"

echo "==== STEP 12: create python 3.6 env ===="
if ! conda env list | grep -q '^py36'; then
  conda create -y -n py36 python=3.6
fi
echo "✅ OK: py36 env ready"

echo "==== STEP 13: activate py36 ===="
conda activate py36
echo "✅ OK: py36 activated"

echo "==== STEP 14: install scientific libs ===="
conda install -y numpy pandas matplotlib scipy cartopy
pip install pyhdf
echo "✅ OK: scientific stack installed"

echo "==== STEP 15: verify imports ===="
python - << 'EOF'
import numpy, pandas, matplotlib, scipy, cartopy, pyhdf
print("ALL IMPORTS SUCCESSFUL")
EOF
echo "✅ OK: imports verified"

echo "==== STEP 16: persist conda auto-load ===="
grep -qxF 'source $HOME/miniconda3/etc/profile.d/conda.sh' ~/.bashrc || \
  echo 'source $HOME/miniconda3/etc/profile.d/conda.sh' >> ~/.bashrc
grep -qxF 'conda activate py36' ~/.bashrc || \
  echo 'conda activate py36' >> ~/.bashrc

echo "🎉 CENTOS SETUP COMPLETED SUCCESSFULLY"
exit 0
'''

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(HOST, username=USER, key_filename=KEY_PATH, timeout=30)

stdin, stdout, stderr = ssh.exec_command(
    f"bash -lc '{SCRIPT_CONDA}'",
    get_pty=True
)

for line in iter(stdout.readline, ""):
    print(line, end="")

err = stderr.read().decode()
if err:
    print(err)
    raise RuntimeError("Conda setup failed")

ssh.close()


### SSH commands for connecting to VM

In [ ]:


print("SSH into Ubuntu (Sciunit author):")
print(f"ssh -i ~/.ssh/P_Key_NASA_DEMO.pem cc@{ubuntu_ip}")

print("SSH into centOS (Sciunit reproducer ):")
print(f"ssh -i ~/.ssh/P_Key_NASA_DEMO.pem cc@{centOS_ip}")


## Install Sciunit on the Ubuntu VM

The following commands must be run **inside the Ubuntu VM terminal**, not in Jupyter.
Sciunit operates at the operating-system level and must be installed in a real shell
environment to correctly capture execution provenance.

open a new terminal, from File >> New >> Terminal

SSH and connect with the remote ubuntu VM using the above "SSH commands for connecting to VM" step, take the first ssh cc@xxx.xxx.xx.xxx execute it in the new opened terminal

### Step 1: Verify Python and pip installation

```bash
python3 --version

pip --version



### (optional if pip not present) Step 2: Install pip

sudo apt update

sudo apt install -y python3-pip

#### check whether pip has installed or not using the following command

pip --version



### Step 3: Install Sciunit

run the following commands

git clone https://github.com/radiant-systems-lab/sciunit.git

cd sciunit

sudo apt install cmake (have to add default yes -y)

git checkout fix-sciunit-copy

pip install . 




### Step 4: Add user bin to PATH

echo 'export PATH=$HOME/.local/bin:$PATH' >> ~/.bashrc

source ~/.bashrc

#### check the version with the following command

sciunit --version

Expected output: sciunit2 0.4.post115.dev43584757

## Install Sciunit on the CentOS VM

The following commands must be run **inside the CentOS VM terminal**, not in Jupyter.
Sciunit operates at the operating-system level and must be installed in a real shell
environment to correctly capture execution provenance.

open a new terminal, from File >> New >> Terminal

SSH and connect with the remote centOS VM using the above "SSH commands for connecting to VM" step, take the second ssh cc@xxx.xxx.xx.xxx and execute it in the new opened terminal



### Step 1: Verify Python and pip installation

```bash
python3 --version

pip --version
```






### (optional if pip not present) Step 2: Install pip  

sudo dnf module enable -y python36

sudo dnf install -y python3-pip

#### check if pip installed with following command

pip --version

### Step 3: Install Sciunit

run the following commands

git clone https://github.com/radiant-systems-lab/sciunit.git

cd sciunit

sudo mkdir -p /etc/yum.repos.d/disabled
sudo mv /etc/yum.repos.d/CentOS-* /etc/yum.repos.d/disabled/

sudo tee /etc/yum.repos.d/CentOS-Vault.repo > /dev/null <<'EOF'
[baseos]
name=CentOS Linux 8 - BaseOS (Vault)
baseurl=http://vault.centos.org/8.5.2111/BaseOS/x86_64/os/
enabled=1
gpgcheck=0

[appstream]
name=CentOS Linux 8 - AppStream (Vault)
baseurl=http://vault.centos.org/8.5.2111/AppStream/x86_64/os/
enabled=1
gpgcheck=0

[extras]
name=CentOS Linux 8 - Extras (Vault)
baseurl=http://vault.centos.org/8.5.2111/extras/x86_64/os/
enabled=1
gpgcheck=0
EOF

sudo dnf clean all
sudo dnf makecache


sudo dnf groupinstall "Development Tools" -y

sudo dnf install -y cmake

`check the version using the following command
cmake --version

`Expected Output: 3.x.x

git checkout fix-sciunit-copy

vi setup.py 

`press i and comment out the line subprocess.call(['apt-get', 'update']) by adding add # at the start

`close the file by pressing ESC and entering the following 
:wq

pip install . 

#### check the version to confirm sciunit installation once done

sciunit --version

Expected output: sciunit2 0.4.post115.dev43584757

## Cleanup 

In [ ]:
ubuntu_vm.delete()
centOS_vm.delete()
l.delete()

print("Cleanup commands prepared (commented out).")


_______________________________________________________________________________